Parcial big data

Importamos librerias de manejo de datos y la base

In [11]:
!pip install pycaret


     |████████████████████████████████| 320 kB 13.8 MB/s 
     |████████████████████████████████| 116 kB 72.6 MB/s 
     |████████████████████████████████| 1.7 MB 45.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 7.1 MB/s 
     |████████████████████████████████| 261 kB 75.8 MB/s 
     |████████████████████████████████| 2.0 MB 61.9 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 16.8 MB 72.3 MB/s 
     |████████████████████████████████| 1.3 MB 55.0 MB/s 
     |████████████████████████████████| 167 kB 79.2 MB/s 
     |████████████████████████████████| 6.8 MB 54.5 MB/s 
     |████████████████████████████████| 690 kB 74.5 MB/s 
     |████████████████████████████████| 3.1 MB 58.6 MB/s 
     |████████████████████████████████| 102 kB 11.4 MB/s 
     |████████████████

In [3]:
import os
import pandas as pd
from statsmodels.robust.scale import mad

df = pd.read_csv("comtradeBEC.csv", sep = ",")


Revisamos la base

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4138 entries, 1 to 4172
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Classification          4138 non-null   object 
 1   Year                    4138 non-null   int64  
 2   Period                  4138 non-null   int64  
 3   Period Desc.            4138 non-null   int64  
 4   Aggregate Level         4138 non-null   int64  
 5   Is Leaf Code            4138 non-null   int64  
 6   Trade Flow Code         4138 non-null   int64  
 7   Trade Flow              4138 non-null   object 
 8   Reporter Code           4138 non-null   int64  
 9   Reporter                4138 non-null   object 
 10  Reporter ISO            4138 non-null   object 
 11  Partner Code            4138 non-null   int64  
 12  Partner                 4138 non-null   object 
 13  Partner ISO             4045 non-null   object 
 14  2nd Partner Code        0 non-null      

Manipulación: Eliminamos el partner "World"

In [4]:
df = df[df['Partner Code'] != 0 & 838]


Eliminamos las que no nos hacen falta

In [5]:
base = df[['Trade Value (US$)', 'Partner Code', 'Period', 'Commodity Code' ]]

Generamos nuevas variables booleanas

In [6]:
principales =  df['Partner Code'] == 842 & 218 & 156 & 591 & 76
hidrocarburos = df['Commodity Code'] == 3
medianTV = df['Trade Value (US$)'] > mad(df['Trade Value (US$)'], c=1)

#hidrocarburos.value_counts()

Las agregamos a la base

In [8]:
base['Principales'] = principales
base['SectorHidro'] = hidrocarburos
base['MedianValue'] = medianTV

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

Las convertimos en categóricas

In [16]:
base['Principales'] = base['Principales'].astype("category")
base['SectorHidro'] = base['SectorHidro'].astype("category")
base['MedianValue'] = base['MedianValue'].astype("category")
base['Partner Code'] = base['Partner Code'].astype("category")
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4138 entries, 1 to 4172
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Trade Value (US$)  4138 non-null   int64   
 1   Partner Code       4138 non-null   category
 2   Period             4138 non-null   int64   
 3   Commodity Code     4138 non-null   int64   
 4   Principales        4138 non-null   category
 5   SectorHidro        4138 non-null   category
 6   MedianValue        4138 non-null   category
dtypes: category(4), int64(3)
memory usage: 159.6 KB


Empezamos el Código de ML

Importamos las funciones

In [13]:
from pycaret.classification import *

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [17]:
exp = setup(data = base, target = 'MedianValue')
logit = create_model("lr")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5069,1.0,1.0,0.5069,0.6728,0.0,0.0
1,0.5069,1.0,1.0,0.5069,0.6728,0.0,0.0
2,0.5069,1.0,1.0,0.5069,0.6728,0.0,0.0
3,0.5069,1.0,1.0,0.5069,0.6728,0.0,0.0
4,0.5069,1.0,1.0,0.5069,0.6728,0.0,0.0
5,0.5103,1.0,1.0,0.5103,0.6758,0.0,0.0
6,0.5087,1.0,1.0,0.5087,0.6743,0.0,0.0
7,0.5087,1.0,1.0,0.5087,0.6743,0.0,0.0
8,0.5087,1.0,1.0,0.5087,0.6743,0.0,0.0


In [18]:
plot_model(logit, plot ="auc")

IntProgress(value=0, description='Processing: ', max=5)

ImportError: ignored

In [19]:
plot_model(logit, plot ="confusion_matrix")

IntProgress(value=0, description='Processing: ', max=5)

ImportError: ignored